In [2]:
import pandas as pd
import numpy as np
import os

In [5]:
# 응모작 파일 이름 가져오기
# 주어진 디렉토리에 있는 항목들의 이름을 담고 있는 리스트를 반환합니다.
file_path = '파일경로'
file_names = os.listdir(file_path)
file_names

In [3]:
## 응모작데이터 셋 불러오기
pome = pd.read_excel("보훈콘텐츠 공모전 응모작관리 및 접수현황예시.xlsx", sheet_name="문예부문 (시)",  engine='openpyxl').fillna(0)
pome["연번"] = pome["연번"].apply(np.int64)
pome

,구분,연번,세부부문,참가구분,마,신청자/팀명,분류
0,문예,1,시 (추모헌시),일반부,시 제목(1),글쓴이(1),독립
1,문예,2,시 (추모헌시),일반부,시 제목(2),글쓴이(2),호국
2,문예,3,시 (추모헌시),초등부,시 제목(3),글쓴이(3),독립
3,문예,4,시 (추모헌시),중고등부,시 제목(4),글쓴이(4),독립
4,문예,5,시 (추모헌시),일반부,시 제목(5),글쓴이(5),독립
...,...,...,...,...,...,...,...
2161,문예,2162,시 (추모헌시),초등부,시 제목(2162),글쓴이(2162),0
2162,문예,2163,시 (추모헌시),초등부,시 제목(2163),글쓴이(2163),0
2163,문예,2164,시 (추모헌시),초등부,시 제목(2164),글쓴이(2164),0
2164,문예,2165,시 (추모헌시),초등부,시 제목(2165),글쓴이(2165),0


In [7]:
#응모작 제목의 공백,특수 문자 제거
pome_name = []
for i in pome["마"]:
    pome_name.append(str(i).rstrip(" ?!"))
pome["마"] = pome_name

0          시 제목(1)
1          시 제목(2)
2          시 제목(3)
3          시 제목(4)
4          시 제목(5)
           ...    
2161    시 제목(2162)
2162    시 제목(2163)
2163    시 제목(2164)
2164    시 제목(2165)
2165    시 제목(2166)
Name: 마, Length: 2166, dtype: object

In [8]:
#기존 파일 형식에 따라서 old_name만들기
rename_list = pd.DataFrame()

# 제목+ 신청자
rename_list["old_name"] = pome["마"].astype(str)+"_"+pome["신청자/팀명"].astype(str)
# 신청자+ 제목+
# rename_list["old_name"] = pome["신청자/팀명"].astype(str)+"_"+pome["마"].astype(str)

# 바꿀 새로운 이름 포멧 만들기
rename_list["new_name"] = "#"+pome["연번"].astype(str)+pome["참가구분"].astype(str)+"_"+pome["마"].astype(str)+"_"+pome["신청자/팀명"].astype(str)
rename_list

,old_name,new_name
0,시 제목(1)_글쓴이(1),#1일반부_시 제목(1)_글쓴이(1)
1,시 제목(2)_글쓴이(2),#2일반부_시 제목(2)_글쓴이(2)
2,시 제목(3)_글쓴이(3),#3초등부_시 제목(3)_글쓴이(3)
3,시 제목(4)_글쓴이(4),#4중고등부_시 제목(4)_글쓴이(4)
4,시 제목(5)_글쓴이(5),#5일반부_시 제목(5)_글쓴이(5)
...,...,...
2161,시 제목(2162)_글쓴이(2162),#2162초등부_시 제목(2162)_글쓴이(2162)
2162,시 제목(2163)_글쓴이(2163),#2163초등부_시 제목(2163)_글쓴이(2163)
2163,시 제목(2164)_글쓴이(2164),#2164초등부_시 제목(2164)_글쓴이(2164)
2164,시 제목(2165)_글쓴이(2165),#2165초등부_시 제목(2165)_글쓴이(2165)


In [9]:
# 기존 파일명과 old_name을 비교해 같을 경우 새로운 파일명으로 변경
for j in file_names:
    # 파일 확장자 뺴기
    raw_name = os.path.splitext(j)[0]
    # 필요하다면 포멧에 맞게 변경
    old_name = raw_name.split("_",1)[-1]
    error = None
    for i in range(len(rename_list["old_name"])):
        try:
            if old_name == rename_list["old_name"][i] :
                new_name = rename_list["new_name"][i]+ os.path.splitext(j)[1]
                src = os.path.join(file_path, j)
                dst = os.path.join(file_path, new_name)
                os.rename(src, dst)
                error = j
        except:
            #엑셀에서 중복된 데이터가 생긴다면 오류 발생, 첫번째 번호를 쓰기 위해서 except사용
            print("엑셀 중복error: ", j)
            
    # 파일명이 바뀌지 않은 파일 보여줌
    if error == None :
        print(j)